In [ ]:
import pandas as pd
import numpy as np

## Читаем данные

In [ ]:
money_train = pd.read_csv('qiwi_money_out_data_train.csv', ';')
money_test = pd.read_csv('qiwi_money_out_data_test.csv',';')
profile_train = pd.read_csv('qiwi_users_profile_data_train.csv',';')
profile_test = pd.read_csv('qiwi_users_profile_data_test.csv',';')

money_train.date_month = list(map(lambda x:int(x[-2:]),money_train.date_month))
money_test.date_month = list(map(lambda x:int(x[-2:]),money_test.date_month))

money_train[:100]

In [ ]:
print (money_train[money_train.user_id==2653].date_month.values)

In [ ]:
categories = np.unique( list(money_train.category.values) + list(money_test.category.values) )
cats ={}
for i in range(categories.shape[0]):
    cats[categories[i]] = i
    
profile_train['university'].fillna('Nouni',inplace=True)
profile_test['university'].fillna('Nouni',inplace=True)
profile_train['graduation_year'].fillna(2018.0,inplace=True)
profile_test['graduation_year'].fillna(2018.0,inplace=True)

universities = np.unique( list(profile_train.university.values) + list(profile_test.university.values) )
unis ={}
for i in range(universities.shape[0]):
    unis[universities[i]] = i

In [ ]:
features = {}
def getX(profiles, payments):
    users = np.unique(payments.user_id.values)
    x = {}
    for user in users:
        x[user] = min(payments[payments.user_id==user].date_month.values)
    users = sorted(x, key=x.get)
    X = np.zeros((np.unique(users).shape[0],3*len(categories)+len(unis)+6))
    y = np.zeros((np.unique(users).shape[0],))
    age = np.zeros((np.unique(users).shape[0],))
    index=0
    print (len(users))
    for user in users:
        pays = payments[payments.user_id==user]
        for cat in categories:
            pay = pays[pays.category==cat]
            X[index,3*cats[cat]] += sum(pay.txn_count.values)
            features[3*cats[cat]] = cat +',payments_count'
            features[3*cats[cat]+1] = cat +',payments_sum'
            X[index,3*cats[cat]+1] += sum(pay.payment.values)
            if sum(pay.txn_count.values)>0:
                X[index,3*cats[cat]+2] = float(sum(pay.payment.values))/sum(pay.txn_count.values)
                features[3*cats[cat]+2] = cat +',payments_average'
        last = 3*len(categories)
        
        X[index,last+unis[profiles[profiles.user_id==user].university.values[0]]] = 1
        features[last+unis[profiles[profiles.user_id==user].university.values[0]]] = profiles[profiles.user_id==user].university.values[0]
        last+=len(unis)
        X[index,last] = profiles[profiles.user_id==user].graduation_year.values[0]
        features[last] = 'Год выпуска'
        last+=1
        X[index,last] = np.unique(pays.category.values).shape[0]
        features[last] = 'Количество различных категорий платежей'
        last+=1
        X[index,last] = np.std(pays.payment.values)
        features[last] = 'Стандартное отклонение суммы платежа за месяц по одной категории'
        last+=1
        X[index,last] = np.unique(pays.date_month.values).shape[0]
        features[last] = 'Количество уникальных месяцев пользования Qiwi'
        last+=1
        X[index,last] = min(pays.date_month.values)
        features[last] = 'Месяц первого платежа'
        last+=1
        X[index,last] = max(pays.date_month.values)
        features[last] = 'Месяц последнего платежа'
        last+=1
        y[index] = int(profiles[profiles.user_id==user].sex.values[0]=='Муж')
        age[index] = profiles[profiles.user_id==user].age.values[0]
        index+=1
        if index%1000==0:
            print (index)
        
    return users,X,y,age


features = {}
def getX(profiles, payments):
    users = np.unique(payments.user_id.values)
    x = {}
    for user in users:
        x[user] = min(payments[payments.user_id==user].date_month.values)
    users = sorted(x, key=x.get)
    X = np.zeros((np.unique(users).shape[0],3*len(categories)+len(unis)+6))
    y = np.zeros((np.unique(users).shape[0],))
    age = np.zeros((np.unique(users).shape[0],))
    index=0
    print (len(users))
    for user in users:
        pays = payments[payments.user_id==user]
        for cat in categories:
            pay = pays[pays.category==cat]
            X[index,3*cats[cat]] += sum(pay.txn_count.values)
            features[3*cats[cat]] = cat +',payments_count'
            features[3*cats[cat]+1] = cat +',payments_sum'
            X[index,3*cats[cat]+1] += sum(pay.payment.values)
            if sum(pay.txn_count.values)>0:
                X[index,3*cats[cat]+2] = float(sum(pay.payment.values))/sum(pay.txn_count.values)
                features[3*cats[cat]+2] = cat +',payments_average'
        last = 3*len(categories)
        
        X[index,last+unis[profiles[profiles.user_id==user].university.values[0]]] = 1
        features[last+unis[profiles[profiles.user_id==user].university.values[0]]] = profiles[profiles.user_id==user].university.values[0]
        last+=len(unis)
        X[index,last] = profiles[profiles.user_id==user].graduation_year.values[0]
        features[last] = 'Год выпуска'
        last+=1
        X[index,last] = np.unique(pays.category.values).shape[0]
        features[last] = 'Количество различных категорий платежей'
        last+=1
        X[index,last] = np.std(pays.payment.values)
        features[last] = 'Стандартное отклонение суммы платежа за месяц по одной категории'
        last+=1
        X[index,last] = np.unique(pays.date_month.values).shape[0]
        features[last] = 'Количество уникальных месяцев пользования Qiwi'
        last+=1
        X[index,last] = min(pays.date_month.values)
        features[last] = 'Месяц первого платежа'
        last+=1
        X[index,last] = max(pays.date_month.values)
        features[last] = 'Месяц последнего платежа'
        last+=1
        y[index] = int(profiles[profiles.user_id==user].sex.values[0]=='Муж')
        age[index] = profiles[profiles.user_id==user].age.values[0]
        index+=1
        if index%1000==0:
            print (index)
        
    return users,X,y,age

def getX_(profiles, payments):
    users = np.unique(payments.user_id.values)
    x = {}
    for user in users:
        x[user] = min(payments[payments.user_id==user].date_month.values)
    users = sorted(x, key=x.get)
    X = np.zeros((np.unique(users).shape[0],3*len(categories)+len(unis)+6))
    #y = np.zeros((np.unique(users).shape[0],))
    #age = np.zeros((np.unique(users).shape[0],))
    index=0
    print (len(users))
    for user in users:
        pays = payments[payments.user_id==user]
        for cat in categories:
            pay = pays[pays.category==cat]
            X[index,3*cats[cat]] += sum(pay.txn_count.values)
            features[3*cats[cat]] = cat +',payments_count'
            features[3*cats[cat]+1] = cat +',payments_sum'
            X[index,3*cats[cat]+1] += sum(pay.payment.values)
            if sum(pay.txn_count.values)>0:
                X[index,3*cats[cat]+2] = float(sum(pay.payment.values))/sum(pay.txn_count.values)
                features[3*cats[cat]+2] = cat +',payments_average'
        last = 3*len(categories)
        
        X[index,last+unis[profiles[profiles.user_id==user].university.values[0]]] = 1
        features[last+unis[profiles[profiles.user_id==user].university.values[0]]] = profiles[profiles.user_id==user].university.values[0]
        last+=len(unis)
        X[index,last] = profiles[profiles.user_id==user].graduation_year.values[0]
        features[last] = 'Год выпуска'
        last+=1
        X[index,last] = np.unique(pays.category.values).shape[0]
        features[last] = 'Количество различных категорий платежей'
        last+=1
        X[index,last] = np.std(pays.payment.values)
        features[last] = 'Стандартное отклонение суммы платежа за месяц по одной категории'
        last+=1
        X[index,last] = np.unique(pays.date_month.values).shape[0]
        features[last] = 'Количество уникальных месяцев пользования Qiwi'
        last+=1
        X[index,last] = min(pays.date_month.values)
        features[last] = 'Месяц первого платежа'
        last+=1
        X[index,last] = max(pays.date_month.values)
        features[last] = 'Месяц последнего платежа'
        last+=1
        #y[index] = int(profiles[profiles.user_id==user].sex.values[0]=='Муж')
        #age[index] = profiles[profiles.user_id==user].age.values[0]
        index+=1
        if index%1000==0:
            print (index)
        
    return users,X

In [ ]:
users, X, Y_train,age = getX(profile_train,money_train)
np.save('X', X)
np.save('users', users)
np.save('Y_train', Y_train)
np.save('age', age)
for k, v in features.items(): 
    print (k, v)
    
users_test, X_test = getX_(profile_test,money_test)

#X_test = (X_test-mean_tr)/std_tr

In [ ]:
import pandas as pd
import numpy as np
X = np.load('X.npy')
ind = np.load('users.npy')
Y_train =  np.load('Y_train.npy')
age =  np.load('age.npy')
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso

In [ ]:
average_importance = {}
for i in range(1):
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
    sumx=0
    print ('Forest ')
    for train_index, test_index in sss.split(X, Y_train):
        Xtr, Ytr, agetr = X[train_index], Y_train[train_index], age[train_index]
        Xval, Yval, ageval = X[test_index], Y_train[test_index], age[test_index]
        if i == 0:
            model = LogisticRegression(class_weight='balanced')
        model.fit(Xtr, Ytr)
        ans = model.predict_proba(Xval)[:, 1].reshape((-1, 1))
        sumx+= (metrics.roc_auc_score(Yval, ans))
        importances= model.coef_
        importnaces = {}
        #print (importances) 
        for i in range(importances[0].shape[0]):
            importnaces[i] = importances[0][i]
        #print (importnaces)
        for f,count in importnaces.items():
            if average_importance.get(f) is None:
                average_importance[f]=0
            average_importance[f] +=float(count)/5
    print (float(sumx)/5)
average = sorted(average_importance, key=average_importance.get)

top_f = average[::-1]
top_f = top_f[:30]
for f in top_f:
    print (features[f],average_importance[f])

print ('Time to women!')
top_f = average
top_f = top_f[:30]
for f in top_f:
    print (features[f],average_importance[f])

In [ ]:
average_importance = {}
for i in range(1):
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)
    sumx=0
    print ('Forest ')
    for train_index, test_index in sss.split(X, Y_train):
        Xtr, Ytr, agetr = X[train_index], Y_train[train_index], age[train_index]
        Xval, Yval, ageval = X[test_index], Y_train[test_index], age[test_index]
        if i == 0:
            model = Lasso()
        model.fit(Xtr, agetr)
        ans = model.predict(Xval)
        sumx+= (np.mean((ans-ageval)**2))**0.5
        importances= model.coef_
        importnaces = {}
        for i in range(importances.shape[0]):
            importnaces[i] = importances[i]
        #print (importnaces)
        for f,count in importnaces.items():
            if average_importance.get(f) is None:
                average_importance[f]=0
            average_importance[f] +=float(count)/5
        print (r2_score(ageval,ans),(np.mean((ans-ageval)**2))**0.5)
    #print (float(sumx)/5)
average = sorted(average_importance, key=average_importance.get)

top_f = average[::-1]
top_f = top_f[:30]
for f in top_f:
    print (features[f],average_importance[f])

print ('Time to women!')
top_f = average
top_f = top_f[:30]
for f in top_f:
    print (features[f],average_importance[f])

In [ ]:
maxx=0.7
maxxx=10
for rate in [.02,.05,.1,.2,.3,.5]:
    average_importance = {}
    sumx=0
    sumx_r=0
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.1)
    for train_index, test_index in sss.split(X, Y_train):
        Xtr, Ytr, agetr = X[train_index], Y_train[train_index], age[train_index]
        Xval, Yval, ageval = X[test_index], Y_train[test_index], age[test_index]
        model = xgb.XGBClassifier(n_estimators=100,learning_rate=rate)
        model.fit(Xtr, Ytr)
        ans = model.predict_proba(Xval)[:, 1].reshape((-1, 1))
        sumx+= (metrics.roc_auc_score(Yval, ans))

        model_r = xgb.XGBRegressor(n_estimators=100,learning_rate=rate)
        model_r.fit(Xtr, agetr)
        ans_r = model_r.predict(Xval)
        sumx_r+= (np.mean((ans_r-ageval)**2))**0.5

    print ((float(sumx)/5),(float(sumx_r)/5))
    if((float(sumx)/5)>maxx):
        print ((metrics.roc_auc_score(Yval, ans)),'Recording gender')
        pol = model.predict_proba(X_test)[:, 1].reshape((-1, 1))
        maxx=(float(sumx)/5)

    if((float(sumx_r)/5)<maxxx):
        print ((np.mean((ans_r-ageval)**2))**0.5,'Recording age')
        age_r = model_r.predict(X_test)
        maxxx=(float(sumx_r)/5)

                           

In [ ]:
res = pd.DataFrame()
res['user_id'] = users_test
res['sex'] = pol
res['age']=age_r
res.to_csv('AlexNet_qiwi_2.csv')